In [3]:
#only needed for installation
#!conda install -c anaconda beautifulsoup4 --yes
#!conda install -c anaconda lxml --yes
#!conda install -c anaconda html5lib --yes
#!conda install -c anaconda requests --yes

import numpy as np 
import pandas as pd 
from bs4 import BeautifulSoup
import requests

# loading a page at a specific url

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_page = requests.get(url).content

# Scrapping a page 
### there a table that have all the data in the page which have the class set as wikitable sortable

In [5]:
soup = BeautifulSoup(html_page,'lxml')
table_postalcode=soup.find('table', class_='wikitable sortable')

postalCodeList=[]
for tableRow in table_postalcode.find_all('tr'):
    entryList=[]
    for tableCol in tableRow.find_all('td'):
        entryList.append(tableCol.text.replace('\n', ''))
    if len(entryList)==3: # only picking with all three enrties
        if entryList[1]!='Not assigned': # leaving the enrties with Borough is set as Not assigned
            if entryList[2]=='Not assigned': #if Neighbourhood is Not assigned then copy the value of Borough 
                entryList[2]=entryList[1]
                postalCodeList.append(entryList)
            else:
                postalCodeList.append(entryList)
headerList=[]
for tableRow in table_postalcode.find_all('tr'):
    for tableheading in tableRow.find_all('th'):
        headerList.append(tableheading.text.replace('\n', ''))

df = pd.DataFrame(postalCodeList, columns = headerList)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


# Joining all the Neighbourhood with same Borough

In [7]:
df1=df.groupby('Postcode')['Neighbourhood'].apply(', '.join)
df=df.drop('Neighbourhood', 1)
df=pd.merge(df, df1, on='Postcode')
df=df.drop_duplicates(keep='last').reset_index(drop=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [8]:
df.shape

(103, 3)

# Part 2

### downloading coordinates from a csv file at http://cocl.us/Geospatial_data and loading into a different dataframe

In [9]:
df_coordinates=pd.read_csv('https://cocl.us/Geospatial_data')
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Renaming the columns names and merging

In [ ]:
df_coordinates=df_coordinates.rename(columns = {"Postal Code": "PostalCode"}) 
df=df.rename(columns = {"Postcode": "PostalCode"})
#merging
